#### Project 4

In [1]:
# uv export --no-hashes > requirements.txt
print("notebook ok")

notebook ok


In [2]:
from research_and_analyst.utils.model_loader import ModelLoader

In [7]:
model_loader = ModelLoader()

{"timestamp": "2025-10-11T22:40:18.771051Z", "level": "info", "event": "GOOGLE_API_KEY loaded from environment"}
{"timestamp": "2025-10-11T22:40:18.774286Z", "level": "info", "event": "GROQ_API_KEY loaded from environment"}
{"timestamp": "2025-10-11T22:40:18.775929Z", "level": "warning", "event": "ASTRA_DB_API_ENDPOINT is missing from environment"}
{"timestamp": "2025-10-11T22:40:18.777968Z", "level": "warning", "event": "ASTRA_DB_APPLICATION_TOKEN is missing from environment"}
{"timestamp": "2025-10-11T22:40:18.779240Z", "level": "warning", "event": "ASTRA_DB_KEYSPACE is missing from environment"}
{"config_keys": ["astra_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-10-11T22:40:18.797108Z", "level": "info", "event": "YAML config loaded"}


In [8]:
llm = model_loader.load_llm()

{"provider": "groq", "model": "llama-3.3-70b-versatile", "timestamp": "2025-10-11T22:40:21.777887Z", "level": "info", "event": "Loading LLM"}


In [9]:
llm.invoke("Hello, how are you?")

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 41, 'total_tokens': 89, 'completion_time': 0.093698505, 'prompt_time': 0.002061287, 'queue_time': 0.092540292, 'total_time': 0.095759792}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9bc8acfb-728a-4076-90fa-bcf5df61d0e7-0', usage_metadata={'input_tokens': 41, 'output_tokens': 48, 'total_tokens': 89})

In [10]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [11]:
def create_analyst(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [12]:
def human_feedback(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [ ]:
def create_analyst(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [13]:
def should_continue(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [15]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [19]:
class Analyst(BaseModel):
    """_summary_
    """
    name: str = Field(..., description="Name of the analyst")
    role: str = Field(..., description="Role of the analyst in the context of the topic")
    affiliation: str = Field(..., description="Primary affiliation of the analyst")
    description: str = Field(..., description="Description of the analyst focus, concerns, and motives")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

In [ ]:
class Perspective(BaseModel):
    analysts: List[Analyst] = Field(..., description="Comprehensive list of analysts with their roles and affiliations.")

In [ ]:
class GenerateAnalystsState(TypedDict):
    """_summary_
    """
    topic: str
    max_analysts: int
    human_analyst_feedback: str
    analysts: List[Analyst]

In [16]:
from IPython.display import display, Markdown, Image

In [17]:
builder = StateGraph()

TypeError: StateGraph.__init__() missing 1 required positional argument: 'state_schema'

In [ ]:
builder.add_node()
builder.add_node()

In [ ]:
builder.edge(START, "create_analyst")
builder.add_edge("create_analyst", "human_feedback")
builder.add_conditional_edges("human_feedback",
                               "should_continue",
                               ["create_analyst",
                                END])

In [18]:
memory = MemorySaver()

In [ ]:
graph = builder.compile(interrupt_before= ["human_feedback"], checkpointer=memory)

In [ ]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

In [ ]:
def should_continue(state):
    """_summary_

    Args:
        state (_type_): _description_
    """
    pass

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
builder = StateGraph()

In [ ]:
builder.add_node()
builder.add_node()

#### Second Workflow

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import os
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [ ]:
tavily_search = TavilySearchResults(tavily_api_key=tavily_api_key)
tavily_search.invoke("langgraph")

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
docs = WikipediaLoader(query="LangGraph").load()
print(docs[0].page_content[:500])

In [ ]:
def search_web(state:InterviewState):
    """
    Retrieve data from the web
    """
    structure_llm = llm.with_structured_output(SearchQuery)